In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk -y

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,266 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,795 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
# Make sure Java is installed
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!java -version

# Set environment variables
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
os.environ["HADOOP_INSTALL"] = "/content/hadoop-3.3.6"
os.environ["HADOOP_PREFIX"] = "/content/hadoop-3.3.6"
os.environ["PATH"] += ":/content/hadoop-3.3.6/bin:/content/hadoop-3.3.6/sbin"


openjdk version "11.0.27" 2025-04-15
OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar -xzf hadoop-3.3.6.tar.gz

--2025-07-10 06:48:53--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz.1’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  23.4MB/s    in 31s     

2025-07-10 06:49:25 (22.6 MB/s) - ‘hadoop-3.3.6.tar.gz.1’ saved [730107476/730107476]



In [ ]:
%%writefile /content/hadoop-3.3.6/etc/hadoop/core-site.xml
<configuration>
  <property>
    <name>fs.defaultFS</name>
    <value>file:///</value>
  </property>
</configuration>


Overwriting /content/hadoop-3.3.6/etc/hadoop/core-site.xml


In [ ]:
%%writefile /content/hadoop-3.3.6/etc/hadoop/hdfs-site.xml
<configuration>
  <property>
    <name>dfs.replication</name>
    <value>1</value>
  </property>
</configuration>


Overwriting /content/hadoop-3.3.6/etc/hadoop/hdfs-site.xml


In [ ]:
!cp /content/hadoop-3.3.6/etc/hadoop/mapred-site.xml.template /content/hadoop-3.3.6/etc/hadoop/mapred-site.xml


cp: cannot stat '/content/hadoop-3.3.6/etc/hadoop/mapred-site.xml.template': No such file or directory


In [ ]:
%%writefile /content/hadoop-3.3.6/etc/hadoop/mapred-site.xml
<configuration>
  <property>
    <name>mapreduce.framework.name</name>
    <value>local</value>
  </property>
</configuration>


Overwriting /content/hadoop-3.3.6/etc/hadoop/mapred-site.xml


In [ ]:
%%writefile mapper.py
#!/usr/bin/env python3
import sys

for line in sys.stdin:
    line = line.strip().lower().replace(".", " ")
    for word in line.split():
        print(f"{word}\t1")


Writing mapper.py


In [ ]:
%%writefile reducer.py
#!/usr/bin/env python3
import sys

current_word = None
current_count = 0

for line in sys.stdin:
    word, count = line.strip().split("\t")
    try:
        count = int(count)
    except:
        continue

    if current_word == word:
        current_count += count
    else:
        if current_word:
            print(f"{current_word}\t{current_count}")
        current_word = word
        current_count = count

if current_word == word:
    print(f"{current_word}\t{current_count}")


Writing reducer.py


In [ ]:
!chmod +x mapper.py reducer.py


In [ ]:
%%writefile sample.txt
Hello Hadoop. Hadoop is cool!
This is a WordCount test.
Hadoop uses MapReduce.
MapReduce splits and reduces.


Writing sample.txt


In [ ]:
!$HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar


/bin/bash: line 1: /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar: Permission denied


In [ ]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
 -input sample.txt \
 -output output \
 -mapper mapper.py \
 -reducer reducer.py


2025-07-10 06:52:44,259 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-07-10 06:52:44,518 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-07-10 06:52:44,518 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-07-10 06:52:44,544 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-07-10 06:52:44,926 INFO mapred.FileInputFormat: Total input files to process : 1
2025-07-10 06:52:44,955 INFO mapreduce.JobSubmitter: number of splits:1
2025-07-10 06:52:45,387 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local344997121_0001
2025-07-10 06:52:45,387 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-07-10 06:52:46,050 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2025-07-10 06:52:46,064 INFO mapreduce.Job: Running job: job_local344997121_0001
2025-07-10 06:52:46,064 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2025-07-1

In [ ]:
!cat output/part-00000


a	1
and	1
cool!	1
hadoop	3
hello	1
is	2
mapreduce	2
reduces	1
splits	1
test	1
this	1
uses	1
wordcount	1
